In [ ]:
######################################################################
######################### 댓글 크롤링 시작 ############################
######################################################################

import urllib.parse as rep
import urllib.request as req
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import gzip
import pandas as pd
import json
import time
import pandas as pd
#from hanspell import spell_checker

In [ ]:


# Header 정보 초기화
opener = req.build_opener()
# User-Agent 정보
#opener.addheaders = [('User-agent', UserAgent())]
# Header 정보 삽입
req.install_opener(opener)

# 쇼핑 댓글 요청 헤더
# https://search.shopping.naver.com/catalog/24178126692?cat_id=50000814&nv_mid=24178126692&NaPm=ct%3Dkmochwcg%7Cci%3D5fbcf90319f44a9577a7ba53d9afec4cf0af44e8%7Ctr%3Dsl%7Csn%3D95694%7Chk%3D180c476dc3ec1764ae9d356ee403773d5f2382ca
# https://search.shopping.naver.com/catalog/14742158672?NaPm=ct%3Dkmyyltb4%7Cci%3D8fb2739c338be12299472ccff160ded9bc19122f%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D0186b9292ab060733115318dcfd65d3aefb527b7

    
# Request
flag = True
pageNum = 1
result = []
count = 0
while flag:
    
    print(len(result))
    # Throttle 설정
    time.sleep(0.7)
    try:
        
        # 쇼핑 댓글 요청 헤더
        hdr = {'authority': 'search.shopping.naver.com', \
        'method': 'GET',\
        'path': '/review?nvMid=14742158672&reviewType=ALL&sort=QUALITY&isNeedAggregation=N&isApplyFilter=N&page='+str(pageNum)+'&pageSize=20',\
        'scheme': 'https',\
        'accept': 'application/json, text/plain, */*',\
        'accept-encoding': 'gzip, deflate, br',\
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',\
        'cache-control': 'no-cache',\
        'pragma': 'no-cache',\
        'referer': 'https://search.shopping.naver.com/catalog/14742158672?NaPm=ct%3Dkmyyltb4%7Cci%3D8fb2739c338be12299472ccff160ded9bc19122f%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D0186b9292ab060733115318dcfd65d3aefb527b7',\
        'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',\
        'sec-ch-ua-mobile': '?0',\
        'sec-fetch-dest': 'empty',\
        'sec-fetch-mode': 'cors',\
        'sec-fetch-site': 'same-origin',\
        'urlprefix': '/api',\
        'user-agent': 'M'}

        url = 'https://search.shopping.naver.com/review?nvMid=14742158672&reviewType=ALL&sort=QUALITY&isNeedAggregation=N&isApplyFilter=N&page='+str(pageNum)+'&pageSize=20'        
        
        reqeust = req.Request(url,headers=hdr)
        res = gzip.decompress(req.urlopen(reqeust).read()).decode('utf-8')
        
        data = json.loads(res)
        result.extend(data['reviews'])
        pageNum = pageNum + 1
        
        
    
    except Exception as e:
        
        # Too many request (429) 일 때 sleep 후 다시 요청
        if e.code == 429:
            print("429 error is occured")
            time.sleep(60)
        else:
            print(str(e))
            print('더 없음')
            flag = False


#res = req.urlopen(reqeust).read().decode('utf-8')



In [ ]:


####################################################################################
######################## 저장된 크롤링 데이터 불러오기 ###############################
####################################################################################

#df = pd.DataFrame(result)
df = pd.read_csv('review_data.csv')
#df.to_csv('review_data.csv',sep=',', na_rep='NaN')
df.describe

In [ ]:
len(result)

In [ ]:
result[6]

In [ ]:
######################################################################
######################### 댓글 크롤링 끝 ##############################
######################################################################

print(df.iloc[0]['content'])

In [ ]:
######################################################################
######################### 댓글 TextRank ##############################
######################################################################
#!pip install kobert-transformers

In [ ]:
#형태소 분석기 Komoran

from konlpy.tag import Komoran

komoran = Komoran()
sent = '쑥 들어간 제품들을 평소에 좋아해서 쑥 화장품으로 대부분 쓰려고 해요 아침 저녁으로 매일 매일 쓰기에 부담이 없고 좋습니다자연약쑥 진정 토너라 자극없이 쓰기에도 좋고 녹차 루이보스 캐모마일 쑥 페퍼민트 로즈마리 유칼리투스잎이 들어간 그린 허브 성분이라 피부 진정에도 도움이 되는거 같아요'

words = komoran.pos(sent, join=True)
print(words )


In [ ]:
# cleasing 함수 
import regex as re
def cleasing(text):
    repl =''
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 자음, 모음 제거
    text = re.sub(pattern= pattern, repl=repl, string=text)
    #pattern = '[^\w\s]' # 특수기호 제거
    pattern = '[^가-히\s]' # 특수기호 제거
    text = re.sub(pattern= pattern, repl=repl, string=text)
    pattern = '<[^>]*>' # html 제거
    text = re.sub(pattern = pattern, repl='',string=text)
    return text
df['length'] = df.content.str.len()
df['clean_content'] = df['content'].map(lambda x: cleasing(x))
df2 = df[ df.length > 30 ] 
df2 = df2['clean_content']
df2


In [ ]:
#from hanspell import spell_checker

check_result = []
index = 0
for i in df2:
    try:
        check_result.append(spell_checker.check(i).checked)

        index = index+1
    except:
        print("error occur!")
        print(i)
        continue

check_result



In [ ]:
##### 교정한 데이터 저장

#pd.DataFrame(check_result).to_csv('review_data_clean.csv',sep=',', na_rep='NaN')

In [ ]:
df2 = pd.read_csv('review_data_clean.csv', header=None)
df2= list(df2[1])

In [ ]:
def komoran_tokenizer(sent):
    try:
        words = komoran.pos(sent, join=True)
        #words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
        words = [w for w in words if ('/NN' in w )]
    except:
        return [0]
    return words

total_analysis = []

for row in df2:
    try:    
        token_list = komoran_tokenizer(row)
        total_analysis.extend(token_list)
    except:
        #pass
        continue

print(total_analysis)

In [ ]:
from textranks.textrank import KeywordSummarizer

keyword_summarizer = KeywordSummarizer(tokenize=komoran_tokenizer, min_count=2, min_cooccurrence=1) 

keyword_extract = keyword_summarizer.summarize(df2, topk=100)


In [ ]:
df_noun = pd.DataFrame(keyword_extract, columns=['words','feq'])
word_sep =  list(df_noun.words.str.split('/'))
print(word_sep[0][1])
df_noun['word'] = [ word_sep[i][0] for i in range(len(word_sep)) ]
tmp_list = []
for i in range(len(word_sep)):
    item = word_sep[i][0]
    tmp_list.append(item)

df_noun['word'] = tmp_list
df_noun = df_noun.drop(['words'], axis=1)
df_noun

In [ ]:
########### 카테고리 정하기 #############

# 명사만 추출한 것
from collections import Counter

count_result = Counter(total_analysis).most_common(300)
print(count_result)

In [ ]:
## 카운팅 결과 DataFrame 으로 만들기
df_noun = pd.DataFrame(count_result, columns=['words','feq'])

word_sep =  list(df_noun.words.str.split('/'))
print(word_sep[0][1])
df_noun['word'] = [ word_sep[i][0] for i in range(len(word_sep)) ]
tmp_list = []
for i in range(len(word_sep)):
    item = word_sep[i][0]
    tmp_list.append(item)

df_noun['word'] = tmp_list
df_noun = df_noun.drop(['words'], axis=1)
df_noun[:50]

In [ ]:
## Transformer 모델 로드하기

import torch
from kobert_transformers import get_kobert_model, get_distilkobert_model
from transformers import BertModel, DistilBertModel

model = BertModel.from_pretrained('monologg/kobert')

#print(model.embeddings.word_embeddings[0])
#model = get_kobert_model()
#model.eval()
model


In [ ]:
unavail_word

In [59]:
#============ Pororo ====================
from pororo import Pororo
cse = Pororo(task="cse", lang="ko")
cse("사용")


ModuleNotFoundError: No module named 'pororo'

In [ ]:
## 임베딩 치환 테스트
from kobert_transformers import get_tokenizer


tokenizer = get_tokenizer()
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")
input_ids = tokenizer.convert_tokens_to_ids(['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]'])
input_ids2 = tokenizer.convert_tokens_to_ids(['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]'])

input_ids2 = tokenizer.convert_tokens_to_ids(['한국'])
input_ids3 = tokenizer.convert_tokens_to_ids(['모델'])
input_ids4 = tokenizer.convert_tokens_to_ids(['구매'])
input_ids5 = tokenizer.convert_tokens_to_ids(['한국'])


#input_ids = torch.LongTensor([input_ids,input_ids2])
input_ids = torch.LongTensor([input_ids2,input_ids3,input_ids4,input_ids5])

print(input_ids)

#attention_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
#token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
#last_hidden_state, pooled_output  = model(input_ids, attention_mask, token_type_ids, return_dict=False)
last_hidden_state, pooled_output  = model(input_ids, return_dict=False)

from scipy import spatial

#print(last_hidden_state[0])
print(last_hidden_state.detach().numpy())
last_hidden_state = last_hidden_state.detach().numpy()
print('--'*20)
result = 1 - spatial.distance.cosine(last_hidden_state[0], last_hidden_state[2])
print(result)
#print(cosine_similarity(last_hidden_state[0],last_hidden_state[1]))


In [ ]:
## 추출한 명사의 임베딩을 추출함

words_list = [[] for _ in range(len(df_noun.word))]
print(type(df_noun.word))

for i,t in df_noun.word.iteritems():
    tmp_list = []
    tmp_list.append(t)
    words_list[i] = tmp_list
#print(words_list)

## 단어가 있는 임베딩만 치환함
index = 0
words_list_not_zero = []
check_word = []
unavail_word = []
for i in words_list:
    #print(tokenizer.convert_tokens_to_ids(i)," : ",df_noun.word[index])
    if tokenizer.convert_tokens_to_ids(i)[0] != 0 :
        words_list_not_zero.append(tokenizer.convert_tokens_to_ids(i))
        check_word.append(df_noun.word[index])
    else:
        print(df_noun.word[index]," is not in vocab")
        unavail_word.append(df_noun.word[index])
    index = index +1 

print(check_word)
print(len(check_word))

input_ids = torch.LongTensor(words_list_not_zero)
last_hidden_state, pooled_output  = model(input_ids, return_dict=False)

sentence_glove = torch.flatten(last_hidden_state, start_dim=1).detach().numpy()
print(sentence_glove.shape)
print(sentence_glove)

In [ ]:
# sklearn
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

# clustering
k = 5
kmeans = KMeans(n_clusters=k, random_state=2021)
y_pred = kmeans.fit_predict(sentence_glove)


# tsne
tsne = TSNE(verbose=1, perplexity=100, random_state=2021)     # perplexity : 유사정도
z = tsne.fit_transform(sentence_glove)

df = pd.DataFrame()
df["y"] = y_pred
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

plt.scatter(z[:,0],z[:,1], c=y_pred, s=50, cmap='viridis')

centers = kmeans.cluster_centers_

In [ ]:
index = 0
result = dict()
for i in y_pred:
    #print(i, " : ",check_word[index] )
    if i in result:
        result[i].append(check_word[index])
    else:
        result[i] = [check_word[index]]
    index = index + 1
for i in result:
    print(result[i])
    print('-'*20)

In [ ]:
# TextRank 중요문장 10개 추출
from textranks.textrank import KeysentenceSummarizer
from konlpy.tag import Komoran

komoran = Komoran()

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)


keysents = summarizer.summarize(list(df2)[:], topk=100)


In [ ]:

for i in keysents:
    print(i)

In [60]:
from pororo import Pororo
summ = Pororo(task="summarization", model="abstractive", lang="ko")


ModuleNotFoundError: No module named 'pororo'